In [ ]:
import numpy as np
from numpy import pi, exp, log10
from numpy.fft import fft, fftshift, fftfreq, ifft
from bokeh.plotting import figure, show, output_notebook
import bokeh.palettes as pl
import panel as pn
pn.extension()
from glob import glob
from tqdm import tqdm 
import os

import utils

import matplotlib.pyplot as plt
import scipy

In [ ]:
fs, x1 = scipy.io.wavfile.read('freebird.wav')
x1 = np.array(x1, dtype=np.float32)
fs, x1.shape

FS = 44100
stft = utils.get_stft(fs=FS)

# cut down waveform size to two minutes centered around the start of the freebird solo
start = 5*60 + 41 #+ 30
end = 7*60 + 41 #- 30
x = x1[start*fs:end*fs]

l = x[:, 0]
r = x[:, 1]
scipy.io.wavfile.write('freebird_cut.wav', fs, np.array([l, r]).T.astype(np.int16))


s = (l + r) / 2 # mono signal
N = len(l)

P_IID, P_IC = utils.encode(stft, l, r)
y_mixed = utils.decode(stft, s, P_IID, P_IC)
l1 = y_mixed.T[:, 0]
r1 = y_mixed.T[:, 1]
scipy.io.wavfile.write('freebird_decoded.wav', fs, y_mixed.T.astype(np.int16))

In [ ]:
import ffmpeg

files = glob('./data/**/**.mp3')

# convert all to wav
for f in tqdm(files):
    out = f.replace('.mp3', '.wav')
    ffmpeg.input(f).output(out).run(overwrite_output=True, quiet=True)

In [ ]:
files = glob('./data/**/**.wav')

# mark any file that won't load as .wav.bad
for f in tqdm(files):
    try:
        fs, l, r = utils.load_wav(f)
    except:
        # print(f)
        os.rename(f, f + '.bad')

# reverse the above
# for f in glob('./data/**/**.wav.bad'):
#    os.rename(f, f.replace('.bad', ''))

In [ ]:
files = glob('./data/**/**.wav')

FS = 44100
stft = utils.get_stft(fs=FS)

# generate PS coding, then save as npz
for f in tqdm(files):
    fs, l, r = utils.load_wav(f)
    N = len(l)
    P_IID, P_IC = utils.encode(stft, l, r)
    out = f + '_ps.npz'
    np.savez_compressed(out, P_IID=P_IID, P_IC=P_IC)

# reverse the above
# for f in glob('./data/**/**.wav_ps.npz'):
#    os.remove(f)

In [ ]:
import os
from glob import glob
# delete all files with _hat.wav
for f in glob('data/**/**_hat.wav'):
    os.remove(f)

FS = 44100
stft = utils.get_stft(fs=FS)

In [ ]:
n_train = 20
n_test = 3
n_frames = 20

train = glob('data/**/**.wav')[:n_train]
test = glob('data/**/**.wav')[n_train:n_train+n_test]
len(train), len(test)

In [ ]:
# train the model

n_samples = 200
features = []
labels = []

from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=1)

for f in tqdm(train):
    if f.find('_hat.wav') > 0: # TODO: really need a more reliable way of doing this
        continue
   
    s, BS, P_IID, P_IC, P, m = utils.file_to_feature_label(f, stft)

    for i in range(n_samples):
        # take a random set of frames from the spectrogram S
        idx = np.random.randint(0, BS.shape[1] - n_frames)

        feature = BS[:, idx:idx+n_frames]
        feature = utils.complex_to_real(feature).flatten()
        features.append(feature)
        
        # the label is the PS parameters of the last frame
        label = P[:, idx+n_frames]
        label = utils.complex_to_real(label)
        labels.append(label)

knn = KNeighborsRegressor(n_neighbors=1).fit(features, labels)
knn

In [ ]:
# test the model
for f in test:
    print(f)
    s, BS, P_IID, P_IC, P, m = utils.file_to_feature_label(f, stft)
    P_hat = np.zeros_like(P)

    features = []
    for i in range(BS.shape[1] - n_frames):
        feature = BS[:, i:i+n_frames]
        feature = utils.complex_to_real(feature).flatten()
        features.append(feature)
    
    labels = knn.predict(features)
    for i in range(len(labels)):
        P_hat[:, i+n_frames] = utils.real_to_complex(labels[i])
    
    P_IID_hat, P_IC_hat = utils.parameters_split(P_hat)
    l_hat, r_hat = utils.decode(stft=stft, s=s, P_IID=P_IID_hat, P_IC=P_IC_hat)
    scipy.io.wavfile.write(f.replace('.wav', '_hat.wav'), FS, np.array([l_hat*m, r_hat*m]).astype(np.int16).T)

    # print mse
    # P_IID /= np.linalg.norm(P_IID)
    # P_IC /= np.linalg.norm(P_IC)
    # P_IID_hat /= np.linalg.norm(P_IID_hat)
    # P_IC_hat /= np.linalg.norm(P_IC_hat)

    mse_IID = np.mean(np.abs(P_IID - P_IID_hat)**2)
    mse_IC = np.mean(np.abs(P_IC - P_IC_hat)**2)
    print(f'mse_IID={mse_IID:.2f}, mse_IC={mse_IC:.2f}', mse_IID/mse_IC)

In [ ]:
import os
from glob import glob
# delete all files with _hat.wav
for f in glob('data/**/**_hat.wav'):
    os.remove(f)

FS = 44100
stft = utils.get_stft(fs=FS)

In [ ]:
n_train = 200
n_test = 50

files = glob('data/**/**.wav')
train = files[:n_train]
test = files[n_train:n_train+n_test]

train_features = np.zeros((n_train, 1294, utils.FREQ_BINS*2))
train_labels = np.zeros((n_train, 1294, utils.FREQ_BINS))

test_features = np.zeros((n_test, 1294, utils.FREQ_BINS*2))
test_labels = np.zeros((n_test, 1294, utils.FREQ_BINS))

for i, f in tqdm(enumerate(train)):
    s, BS, P_IID, P_IC, P, m = utils.file_to_feature_label(f, stft)
    train_features[i] = utils.complex_to_real(BS).T
    train_labels[i] = P_IC.real.T

for i, f in tqdm(enumerate(test)):
    s, BS, P_IID, P_IC, P, m = utils.file_to_feature_label(f, stft)
    test_features[i] = utils.complex_to_real(BS).T
    test_labels[i] = P_IC.real.T

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Input, LSTM, Conv1D

model = Sequential()
model.add(Input(shape=(None, utils.FREQ_BINS*2)))

model.add(SimpleRNN(units=utils.FREQ_BINS, return_sequences=True)) #activation=activation[0]))
model.add(SimpleRNN(units=utils.FREQ_BINS*2, return_sequences=True)) #activation=activation[0]))
model.add(SimpleRNN(units=utils.FREQ_BINS*8, return_sequences=True)) #activation=activation[0]))
model.add(SimpleRNN(units=utils.FREQ_BINS*32, return_sequences=True)) #activation=activation[0]))
model.add(SimpleRNN(units=utils.FREQ_BINS*32, return_sequences=True)) #activation=activation[0]))
model.add(SimpleRNN(units=utils.FREQ_BINS*8, return_sequences=True)) #activation=activation[0]))
model.add(SimpleRNN(units=utils.FREQ_BINS*2, return_sequences=True)) #activation=activation[0]))
model.add(SimpleRNN(units=utils.FREQ_BINS, return_sequences=True)) #activation=activation[0]))
# model.add(LSTM(units=utils.FREQ_BINS*2, return_sequences=True)) #activation=activation[1]))
model.compile(loss='mean_squared_error', optimizer='adam')

model.summary()


In [ ]:
model.fit(train_features, train_labels, epochs=10, batch_size=1)

In [ ]:
# model.evaluate(train_features, train_labels, batch_size=1)
# model.evaluate(test_features, test_labels, batch_size=1)
print(np.abs(train_labels[0]).round(2)[:1])
print(np.abs(model.predict(train_features[:1])[0]).round(2)[:1])

In [ ]:
for f in test:
    print(f)
    s, BS, P_IID, P_IC, P, m = utils.file_to_feature_label(f, stft)
    P_hat = np.zeros_like(P)

    feature = utils.complex_to_real(BS).T
    # label = utils.complex_to_real(P).T
    
    P_hat = utils.real_to_complex(model.predict(np.array([feature]))[0].T)
    P_IID_hat, P_IC_hat = utils.parameters_split(P_hat)
    l_hat, r_hat = utils.decode(stft=stft, s=s, P_IID=P_IID_hat, P_IC=P_IC_hat)
    scipy.io.wavfile.write(f.replace('.wav', '_hat.wav'), FS, np.array([l_hat*m, r_hat*m]).astype(np.int16).T)

    # print mse
    # P_IID /= np.linalg.norm(P_IID)
    # P_IC /= np.linalg.norm(P_IC)
    # P_IID_hat /= np.linalg.norm(P_IID_hat)
    # P_IC_hat /= np.linalg.norm(P_IC_hat)

    mse_IID = np.mean(np.abs(P_IID - P_IID_hat)**2)
    mse_IC = np.mean(np.abs(P_IC - P_IC_hat)**2)
    print(np.mean(np.abs(P_IID_hat)), np.mean(np.abs(P_IC_hat)), sep='\t')
    print(np.mean(np.abs(P_IID)), np.mean(np.abs(P_IC)), sep='\t')
    print(f'mse_IID={mse_IID:.2f}, mse_IC={mse_IC:.2f}', mse_IID/mse_IC)

In [ ]:
X = 10**(np.abs(P_IID)/10)

This creates the $\mathbf{B}$ matrix which consists of 1s and 0s and sums the 2049 FFT bins into 34 frequency bands

The cross spectrogram is defined as:
$$\rho(\mathbf{X}, \mathbf{Y}) = \mathbf{B}(\mathbf{X} \times \mathbf{Y}^*)$$
where $\times$ denotes element-wise multiplication, and $*$ denotes element-wise complex conjugation.

In [ ]:
from bokeh import palettes as pl
from bokeh.models import LogColorMapper, ColorBar

m = LogColorMapper(palette=pl.Inferno256, low=X.min(), high=X.max())

p = figure(width=1500, height=700, title='Spectrogram', x_axis_label='Time (s)', y_axis_label='Frequency (kHz)')
p.min_border=0

p.image(image=[X], x=0, y=0, dw=X.shape[1]*int(4096*(1-0.75))/fs, dh=fs/2/1e3, color_mapper=m)
p.add_layout(ColorBar(color_mapper=m), 'right')

show(p)